In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement

# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [3]:
import pandas as pd

# Define the file path
file_path = r"D:\feature.csv"

# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')

columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
print(data.head(1))


   Feature 0  Feature 1  Feature 2  Feature 3  Feature 4  Feature 5  \
0        0.0        0.0        0.0        0.0        0.0        0.0   

   Feature 6  Feature 7  Feature 8  Feature 9  ...  Feature 2038  \
0        0.0        0.0        0.0        0.0  ...           0.0   

   Feature 2039  Feature 2040  Feature 2041  Feature 2042  Feature 2043  \
0           0.0      0.009121           0.0           0.0           0.0   

   Feature 2044  Feature 2045  Feature 2046  Feature 2047  
0           0.0      3.285227           0.0           0.0  

[1 rows x 2048 columns]


In [4]:
from PIL import Image
import numpy as np
ft = model.predict(x_train)


In [5]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 300
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [6]:
from tqdm import tqdm

combo_list = []  # Initialize combo_list

# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    # Retrieve shortest indices and select corresponding rows from df
    shortest_indices = np.argsort(distances)[:301]
    selected_rows = df.loc[shortest_indices]

    # Extract values from the first column and add them to the combo_list
    first_column_values = selected_rows.iloc[:, 0].tolist()
    
    k = 0
    
    for element in first_column_values:
        if element not in combo_list:
            combo_list.append(element)
            break

# Printing the combo_list
print(combo_list)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:13<00:00,  4.09it/s]

['D:/data/imagenet\\n03450230\\', 'D:/data/imagenet\\n04273569\\', 'D:/data/imagenet\\n07836838\\', 'D:/data/imagenet\\n02510455\\', 'D:/data/imagenet\\n04146614\\', 'D:/data/imagenet\\n02085782\\', 'D:/data/imagenet\\n02328150\\', 'D:/data/imagenet\\n01491361\\', 'D:/data/imagenet\\n01776313\\', 'D:/data/imagenet\\n04392985\\', 'D:/data/imagenet\\n03590841\\', 'D:/data/imagenet\\n02667093\\', 'D:/data/imagenet\\n03344393\\', 'D:/data/imagenet\\n04525305\\', 'D:/data/imagenet\\n03769881\\', 'D:/data/imagenet\\n03459775\\', 'D:/data/imagenet\\n07831146\\', 'D:/data/imagenet\\n03197337\\', 'D:/data/imagenet\\n04591157\\', 'D:/data/imagenet\\n04069434\\', 'D:/data/imagenet\\n03393912\\', 'D:/data/imagenet\\n02018795\\', 'D:/data/imagenet\\n03692522\\', 'D:/data/imagenet\\n02100877\\', 'D:/data/imagenet\\n03494278\\', 'D:/data/imagenet\\n02009912\\', 'D:/data/imagenet\\n03127747\\', 'D:/data/imagenet\\n03180011\\', 'D:/data/imagenet\\n10148035\\', 'D:/data/imagenet\\n03314780\\', 'D:/data/

In [7]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(combo_list, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [8]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['D:/data/imagenet\\n03450230\\', 'D:/data/imagenet\\n04273569\\', 'D:/data/imagenet\\n07836838\\', 'D:/data/imagenet\\n02510455\\', 'D:/data/imagenet\\n04146614\\', 'D:/data/imagenet\\n02085782\\', 'D:/data/imagenet\\n02328150\\', 'D:/data/imagenet\\n01491361\\', 'D:/data/imagenet\\n01776313\\', 'D:/data/imagenet\\n04392985\\', 'D:/data/imagenet\\n03590841\\', 'D:/data/imagenet\\n02667093\\', 'D:/data/imagenet\\n03344393\\', 'D:/data/imagenet\\n04525305\\', 'D:/data/imagenet\\n03769881\\', 'D:/data/imagenet\\n03459775\\', 'D:/data/imagenet\\n07831146\\', 'D:/data/imagenet\\n03197337\\', 'D:/data/imagenet\\n04591157\\', 'D:/data/imagenet\\n04069434\\', 'D:/data/imagenet\\n03393912\\', 'D:/data/imagenet\\n02018795\\', 'D:/data/imagenet\\n03692522\\', 'D:/data/imagenet\\n02100877\\', 'D:/data/imagenet\\n03494278\\', 'D:/data/imagenet\\n02009912\\', 'D:/data/imagenet\\n03127747\\', 'D:/data/imagenet\\n03180011\\', 'D:/data/imagenet\\n10148035\\', 'D:/data/i